In [2]:
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix, classification_report




OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\SIDDHAARTH\Desktop\Virtual Env for Pyhton\.venv\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [ ]:
base = r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data"

angry   = fr"{base}\angry"
disgust = fr"{base}\disgust"
fear    = fr"{base}\fear"
happy   = fr"{base}\happy"
neutral = fr"{base}\neutral"
sad     = fr"{base}\sad"


def show_images_from_folder(folder_path, n=12, cols=4, figsize=(10,8)):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
             if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    if not files:
        print("No images found in:", folder_path)
        return
    
    files = files[:n]  # take first n
    
    rows = (n + cols - 1) // cols
    plt.figure(figsize=figsize)

    for i, path in enumerate(files):
        img = Image.open(path).convert('RGB')
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(folder_path).capitalize())
        print(img.size)

    plt.tight_layout()
    plt.show()


In [ ]:
show_images_from_folder(angry, n=4)
show_images_from_folder(happy, n=4)
show_images_from_folder(neutral, n=4)



In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([ 
    transforms.ToTensor()         
])
dataset = datasets.ImageFolder(r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data", transform=transform)




In [ ]:
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
dev_size = len(dataset) - test_size - train_size

train_ds , test_ds , dev_ds = random_split(
    dataset,
    [train_size , test_size ,dev_size],
    generator=torch.Generator().manual_seed(42) 
)



In [ ]:
train_loader_temp = DataLoader(train_ds , batch_size = 64 ,shuffle=True)

std_train = 0
mean_train = 0
total_train = 0

for images , _ in tqdm(train_loader_temp):
    imgs = images.view(images.size(0) , images.size(1) , -1)
    mean_train += imgs.mean(2).sum(0)
    std_train += imgs.std(2).sum(0)
    total_train += imgs.shape[0]

mean_train /= total_train
std_train /= total_train



In [ ]:
dev_loader_temp = DataLoader(dev_ds , batch_size = 64 ,shuffle=True)

std_dev = 0
mean_dev = 0
total_dev = 0

for images , _ in tqdm(dev_loader_temp):
    imgs = images.view(images.size(0) , images.size(1) , -1)
    mean_dev += imgs.mean(2).sum(0)
    std_dev += imgs.std(2).sum(0)
    total_dev += imgs.shape[0]

mean_dev /= total_dev
std_dev /= total_train



In [ ]:
print(mean_train , std_train)
print(mean_dev ,  std_dev)

In [ ]:
mean_list_train = mean_train.tolist()
std_list_train = std_train.tolist() 

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_list_train , std_list_train)
])



In [ ]:
mean_list_dev = mean_dev.tolist()
std_list_dev = std_dev.tolist()

dev_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_list_dev , std_list_dev)
])

In [ ]:
dataset_normalized = datasets.ImageFolder(r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data")

train_ds_final = torch.utils.data.Subset(
    datasets.ImageFolder(r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data", transform=train_transform), train_ds.indices
)

dev_ds_final = torch.utils.data.Subset(
    datasets.ImageFolder(r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data", transform=dev_transform), dev_ds.indices
)

test_ds_final = torch.utils.data.Subset(
    datasets.ImageFolder(r"C:\Users\Siddharth Sharma\OneDrive\Desktop\Video project\processed_data", transform=dev_transform), test_ds.indices
)


In [ ]:
train_loader = DataLoader(train_ds_final, batch_size=64, shuffle=True)
val_loader   = DataLoader(dev_ds_final, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_ds_final, batch_size=64, shuffle=False)

In [ ]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))


IT means that there is 623 batches of the train set and each batch has 64 images!!

In [ ]:

device = torch.device("cpu")
print("Using device:", device)


In [ ]:
class Training_loop(nn.Module):
    def __init__(self , num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3 , 16 , 3 , padding=1)
        self.conv2 = nn.Conv2d(16 , 32 , 3 , padding=1)
        self.conv3 = nn.Conv2d(32 , 64 , 3 , padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(64 * 12 * 12, num_classes)

    def forward(self , x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        #this is the 1st Conv layer!!
        
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)
        #this is the 2nd Conv layer!!
                
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.pool(x)
        #this is the 3rd Conv layer!!

        return self.fc(x)
    


In [ ]:

try:
    classes = train_loader.dataset.dataset.classes 
    num_classes = len(classes)
    print("Inferred classes from ImageFolder:", classes)
except Exception:
   
    labels = []
    for _, y in train_loader.dataset:
        labels.append(int(y))
    num_classes = int(max(labels)) + 1
    classes = [str(i) for i in range(num_classes)]
    print("Inferred num_classes from data:", num_classes)


In [ ]:

model = Training_loop(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.1)
print("Model params:", sum(p.numel() for p in model.parameters()))
